In [23]:
import calcOneDay
import getDays
from datetime import datetime, timedelta
import calcTimeNow
import daysAndDates
import logging

#
# Set up some logging
#

'''
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')

file_handler = logging.FileHandler('/home/ec2-user/davisComplete99.log')
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)
'''

# Calculate the time and date for end of day calculations

xy = calcOneDay.calcOneDay()
start, end = (xy[0], xy[1])
dayInfo = daysAndDates.daysAndDates()

month, month_num, date, year = dayInfo[0], dayInfo[1], dayInfo[2], dayInfo[3]
yesterday = int(dayInfo[4])
nextDay = int(dayInfo[5])

In [24]:
import collections
import hashlib
import hmac
import time
from datetime import datetime
import requests
import json
import dataFile
import logging


parameters = {
  "api-key": "vy8jbrjsxlbwgojepq3vfyfqfywyhvbd", 
  "api-secret": "sdqfm6wdfy9w0pqp2vdka38o6b4vcsvc",
  "station-id": 81211, 
  "end-timestamp": end,
  "start-timestamp": start,
  "t": int(time.time())
}

parameters = collections.OrderedDict(sorted(parameters.items()))

#for key in parameters:
#    logger.info("Parameter name: \"{}\" has value \"{}\"".format(key, parameters[key]))

apiSecret = parameters["api-secret"];
parameters.pop("api-secret", None);

data = ""
for key in parameters:
    data = data + key + str(parameters[key])

#logger.info('Data string to hash is: \"{}\"'.format(data))   

"""
Calculate the HMAC SHA-256 hash that will be used as the API Signature.
"""
apiSignature = hmac.new(
  apiSecret.encode('utf-8'),
  data.encode('utf-8'),
  hashlib.sha256
).hexdigest()

"""
Let's see what the final API Signature looks like.
"""

#logger.info('API Signature is: \"{}\"'.format(apiSignature))

# Building the URL to get the station

first_part = ('https://api.weatherlink.com/v2/historic/81211?')
api_key = ('api-key=vy8jbrjsxlbwgojepq3vfyfqfywyhvbd')
add_apisig = ('&api-signature=')
add_t = ('&t='+ str(int(time.time())))

start1 = "&start-timestamp=" + start
end1 = "&end-timestamp=" + end

URLfinal = (first_part + api_key + add_t + start1 + end1 + add_apisig + apiSignature)

r =  requests.get(URLfinal)
davisAPI = (r.json())
print(davisAPI)

{'station_id_uuid': '245f6069-3309-4f78-aa0e-8f97d4e5d6ec', 'sensors': [{'lsid': 274372, 'data': [{'temp_in_lo_at': 1701734904, 'arch_int': 60, 'temp_in_hi': 81.9, 'temp_in_hi_at': 1701734934, 'hum_in_hi': 34.4, 'temp_in_last': 81.9, 'temp_in_lo': 81.8, 'hum_in_lo': 34.4, 'hum_in_last': 34.4, 'dew_point_in': 51.1, 'hum_in_lo_at': 1701734904, 'tz_offset': -18000, 'heat_index_in': 80.3, 'hum_in_hi_at': 1701734904, 'ts': 1701734940}, {'temp_in_lo_at': 1701734994, 'arch_int': 60, 'temp_in_hi': 81.8, 'temp_in_hi_at': 1701734964, 'hum_in_hi': 34.6, 'temp_in_last': 81.7, 'temp_in_lo': 81.7, 'hum_in_lo': 34.4, 'hum_in_last': 34.6, 'dew_point_in': 51.1, 'hum_in_lo_at': 1701734964, 'tz_offset': -18000, 'heat_index_in': 80.2, 'hum_in_hi_at': 1701734994, 'ts': 1701735000}, {'temp_in_lo_at': 1701735024, 'arch_int': 60, 'temp_in_hi': 81.7, 'temp_in_hi_at': 1701735024, 'hum_in_hi': 34.5, 'temp_in_last': 81.7, 'temp_in_lo': 81.7, 'hum_in_lo': 34.5, 'hum_in_last': 34.5, 'dew_point_in': 51, 'hum_in_lo_a

In [25]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import sqlalchemy
from dateutil.tz import tzutc, tzlocal
import pytz

a = davisAPI['sensors']    
b = a[1]
c = (b['data'])

df = pd.DataFrame(c) 
df.rename(columns = {'ts':'timestamp'}, inplace = True)

timezone = pytz.timezone("America/New_York")
df['timeGroup'] = pd.to_datetime(df['timestamp'], unit='s')
df['timeGroup'] = df['timeGroup'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
df['localTime'] = df['timeGroup'].dt.strftime('%I:%M %p')

df = df.loc[:,['timestamp', 'temp_hi', 'temp_hi_at','temp_lo', 'temp_lo_at', 'rainfall_in', 'dew_point_hi', 'dew_point_lo',  'rain_rate_hi_in', 'rain_rate_hi_at', 'timeGroup', 'localTime']]

'''
database_username = 'chuckwx'
database_password = 'jfr716!!00'
database_ip       = '3.135.162.69'
database_name     = 'davisInfo'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), connect_args={'connect_timeout': 30})
df.to_sql(con=database_connection, name='davisTestB', if_exists='append')
'''

"\ndatabase_username = 'chuckwx'\ndatabase_password = 'jfr716!!00'\ndatabase_ip       = '3.135.162.69'\ndatabase_name     = 'davisInfo'\ndatabase_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.\n                                               format(database_username, database_password, \n                                                      database_ip, database_name), connect_args={'connect_timeout': 30})\ndf.to_sql(con=database_connection, name='davisTestB', if_exists='append')\n"

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import math

max_temp  = (df.sort_values(by='temp_hi', ascending=False))
max_T = max_temp.iloc[:1]
max_T_time = int(max_T['temp_hi_at'])
maxT = max_T['temp_hi'].values[0]
maxT = round(maxT)

min_temp  = (df.sort_values(by='temp_lo', ascending=True))
min_T = min_temp.iloc[:1]
min_T_time = int(min_T['temp_lo_at'])
minT = min_T['temp_lo'].values[0]
minT = round(minT)

dew_max = (df.sort_values(by='dew_point_hi', ascending=False))
dew_max1 = dew_max.iloc[:1]
dewMax = dew_max1['dew_point_hi'].values[0]
dewMaxT = round(dewMax)

dew_min = (df.sort_values(by='dew_point_lo', ascending=True))
dew_min1 = dew_min.iloc[:1]
dewMin = dew_min1['dew_point_lo'].values[0]
dewMinT = round(dewMin)

rain = df['rainfall_in'].sum()
avgTemp = math.ceil((int(maxT + minT)/2))

hdd = (65 - avgTemp)
if hdd < 0:
    hdd = 0
cdd = (avgTemp - 65)
if cdd < 0:
    cdd = 0  

In [27]:
import sqlalchemy
import mysql.connector
import sqlite3

df2 = pd.DataFrame(columns = ['Year', 'Month', 'Date', 'High', 'Low', 'Average', 'HDD', 'CDD', 'Rainfall', 'Max_Dew_Point', 'Min_Dew_Point'])
newRow = pd.DataFrame({'Year': year, 'Month': month_num, 'Date': yesterday, 'High': maxT, 'Low': minT, 'Average': avgTemp, 'HDD': hdd, 'CDD': cdd, 'Rainfall' : rain, 'Max_Dew_Point': dewMaxT, 'Min_Dew_Point': dewMinT}, index = [yesterday])
df2 = pd.concat([newRow, df2]).reset_index(drop = True)

'''
database_username = 'chuckwx'
database_password = 'jfr716!!00'
database_ip       = '3.135.162.69'
database_name     = 'davisf6'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), connect_args={'connect_timeout': 30})
df2.to_sql(con=database_connection, name='davisF6A', if_exists='append', index = False)
'''

"\ndatabase_username = 'chuckwx'\ndatabase_password = 'jfr716!!00'\ndatabase_ip       = '3.135.162.69'\ndatabase_name     = 'davisf6'\ndatabase_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.\n                                               format(database_username, database_password, \n                                                      database_ip, database_name), connect_args={'connect_timeout': 30})\ndf2.to_sql(con=database_connection, name='davisF6A', if_exists='append', index = False)\n"